In [1]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=1)
client

c:\Users\ls2236\AppData\Local\anaconda3\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50777 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:50777/status,
Dashboard: http://127.0.0.1:50777/status,Workers: 1
Total threads: 4,Total memory: 63.25 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50779,Workers: 1
Dashboard: http://127.0.0.1:50777/status,Total threads: 4
Started: Just now,Total memory: 63.25 GiB
Comm: tcp://127.0.0.1:50786,Total threads: 4
Dashboard: http://127.0.0.1:50787/status,Memory: 63.25 GiB
Nanny: tcp://127.0.0.1:50782,


In [2]:
import time
import random

def costly_simulation(list_param):
    time.sleep(random.random())
    return sum(list_param)

In [3]:
%time costly_simulation([1, 2, 3, 4])


CPU times: total: 0 ns
Wall time: 312 ms


10

In [4]:
import pandas as pd
import numpy as np

input_params = pd.DataFrame(np.random.random(size=(500, 4)),
                            columns=['param_a', 'param_b', 'param_c', 'param_d'])
input_params.head()

,param_a,param_b,param_c,param_d
0,0.361660,0.951425,0.039748,0.449094
1,0.316568,0.929531,0.014773,0.486726
2,0.310037,0.867359,0.020525,0.590873
3,0.547056,0.434081,0.129115,0.118985
4,0.746568,0.313858,0.520813,0.003265


In [5]:
results = []


In [6]:
%%time
for parameters in input_params.values[:10]:
    result = costly_simulation(parameters)
    results.append(result)

CPU times: total: 15.6 ms
Wall time: 3.53 s


In [7]:
import dask
lazy_results = []

In [8]:
%%time

for parameters in input_params.values[:10]:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)

CPU times: total: 0 ns
Wall time: 0 ns


In [9]:
lazy_results[0]


Delayed('costly_simulation-34613953-25c1-4173-958a-efa58bededee')

In [11]:
%time dask.compute(*lazy_results)


CPU times: total: 15.6 ms
Wall time: 1.51 s


(1.801926951450851,
 1.7475979749585542,
 1.7887941949027732,
 1.2292374187287503,
 1.584504209808526,
 2.000369940165191,
 1.9412811647349322,
 1.1208375362134584,
 2.3921296831691334,
 2.2188421776675984)

In [12]:
lazy_results = []

for parameters in input_params.values:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)

futures = dask.persist(*lazy_results)  # trigger computation in the background

In [13]:
client.shutdown()

2024-09-26 19:44:32,595 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:50786' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'costly_simulation-73aed4ce-82d2-4d35-9e2e-ebbb82fd5908', 'costly_simulation-f2f38b79-b3aa-46d6-b135-3f757a25723d', 'costly_simulation-88a7a6da-0c1d-47fc-8a1e-9eb8ab6d8667', 'costly_simulation-1a740d9a-ecfc-4021-b4a9-e9df6c31abed', 'costly_simulation-d83e0815-02c2-4f3e-8fdf-70e1618bb47b', 'costly_simulation-5359393a-92f5-4e8e-9151-906fb390dfb6', 'costly_simulation-78d387ff-3ed7-4742-8d75-6da9b093d57e', 'costly_simulation-4cc71573-215f-4dd4-9f0d-007c7de9f774', 'costly_simulation-a94e3b09-97a7-4178-9684-2e5fcf1f4f8a', 'costly_simulation-ba3bac46-6dd5-45a3-99c6-56520b758309', 'costly_simulation-aeba83b8-523f-46b4-99a8-cefc6b547caf', 'costly_simulation-c4028395-bd9b-4cca-9463-694d49512c37', 'costly_simulation-c09aa621-edbb-4c6f-b760-d83486ca0d0a', 'costly_simulation-6d2cbeb3-16ae-427e-b8ec-6a73513aa1db'